## CREATORS OF THIS FILE
Merlijn Sevenhuijsen
Alessandro Meroli


In [81]:
# Import libraries
import os
import requests
import json
import pandas as pd
import numpy as np
from retry_requests import retry
from helper import tomtom_api_params, get_weather_data, handle_incident, get_incident_details, handle_weather_data

In [83]:
# Dictionary of all parameters for the API
api_params_incidents = tomtom_api_params()

In [84]:
# Load the incidents csv if it exists
if os.path.isfile('incidents.csv'):
    df_incidents = pd.read_csv('incidents.csv')
else:
    df_incidents = pd.DataFrame(columns=['id', 'type', 'magnitudeOfDelay', 'startTime', 'endTime', 'description', 'code', 'iconCategory', 'longitude', 'latitude', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'apparent_temperature', 'precipitation', 'rain', 'snowfall', 'snow_depth', 'weather_code', 'surface_pressure', 'et0_fao_evapotranspiration', 'vapour_pressure_deficit', 'wind_speed_10m', 'soil_temperature_0_to_7cm', 'is_day', 'sunshine_duration'])
    df_incidents.set_index('id', inplace=True)

In [115]:
# Function that handles an incident call
def handle_incident_call(incidents, df_incidents, weather_data):
    # Create a list that will contain all the incidents
    list_incidents = []

    # For each active incident
    for i in range(len(incidents)):
        # Check if the incident is already in the dataframe
        if  incidents[i]['properties']['id'] in df_incidents.index.values:
            print('Incident already in dataframe')
            continue

        # Check if the incident is active
        # if incidents[i]['properties']['endTime'] == None:
        #     print('Incident has no ending time')
        #     continue

        # Get the date
        date = incidents[i]['properties']['startTime'][:10]

        # Create a dictionary with the incident details
        dict_incident = handle_incident(incidents[i], weather_data[date])

        # Add the incident to the dataframe
        list_incidents.append(dict_incident)

    # Create a second dataframe with the new values
    df_incidents_2 = pd.DataFrame(list_incidents)
    if len(df_incidents_2) > 0:
        df_incidents_2.set_index('id', inplace=True)

    # print the amount of new incidents
    print(f"New incidents: {len(df_incidents_2)}")
    
    # Concatenate the new incidents to the old dataframe
    df_incidents = pd.concat([df_incidents, df_incidents_2], sort=True)
    return df_incidents

In [87]:
# Function that gets all different dates in the response
def get_dates(incidents):
    dates = []
    for i in range(len(incidents)):
        date = incidents[i]['properties']['startTime'][:10]
        if date not in dates:
            dates.append(date)
    return dates

In [119]:
# The position of Stockholm
lon = 18.0687
lat = 59.3294

# Add incidents to the dataframe based on t, divided over 5 steps
t =1703251660
all_incidents = get_incident_details(api_params_incidents, t)
all_incidents = all_incidents['incidents']

# Filter out only the incidents that are have an end time
# all_incidents = [incident for incident in all_incidents if incident['properties']['endTime'] != None]

# Also filter out incidents that are not already processed in incidents.csv
all_incidents = [incident for incident in all_incidents if incident['properties']['id'] not in df_incidents.index.values]

# Get all dates
dates = get_dates(all_incidents)

# A dictionary that for each day contains the weather data
weather_data = {}
for date in dates:
    # Weather data for the current date
    hourly_weather = handle_weather_data(get_weather_data(lon, lat, date))
    
    # Add the weather data to the dictionary
    weather_data[date] = hourly_weather

# Filter out all the incidents that are not in the weather data
# all_incidents = [incident for incident in all_incidents if incident['properties']['startTime'][:10] in weather_data]

# Handle the incidents
df_incidents = handle_incident_call(all_incidents, df_incidents, weather_data)

# Save the dataframe to a csv file
df_incidents.to_csv('incidents.csv', index=True)


New incidents: 0


In [113]:
print(hourly_weather)

                  date  temperature_2m  relative_humidity_2m  dew_point_2m  \
0  2023-09-10 00:00:00       16.014999             95.006775     15.214999   
1  2023-09-10 01:00:00       15.565000             97.153915     15.115000   
2  2023-09-10 02:00:00       15.315000             97.148483     14.865000   
3  2023-09-10 03:00:00       15.214999             97.459824     14.815000   
4  2023-09-10 04:00:00       15.065000             97.456879     14.665000   
5  2023-09-10 05:00:00       15.315000             97.148483     14.865000   
6  2023-09-10 06:00:00       16.715000             91.745285     15.365000   
7  2023-09-10 07:00:00       18.415001             84.826599     15.815000   
8  2023-09-10 08:00:00       20.065001             77.527977     16.014999   
9  2023-09-10 09:00:00       21.315001             73.169731     16.315001   
10 2023-09-10 10:00:00       22.465000             70.412918     16.815001   
11 2023-09-10 11:00:00       23.165001             68.783325    